In [1]:
# Import modules
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras.optimizers import SGD
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
import random
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten

/Users/Michael/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# Get direcotry path for each image
def GetImagePathList(data_dir):
    image_path_list = []
    for foldername in os.listdir(data_dir):
        if not foldername.startswith('.'):
            path = data_dir + foldername + '/'
            for filename in os.listdir(path):
                if not filename.startswith('.'):
                    image_path = path + filename
                    image_path_list.append(image_path)
    return image_path_list

image_path_list = GetImagePathList('data/')

print(image_path_list[:4]) # first 4 elements
print(image_path_list[-4:]) # last 4 elements

['data/sky/IMG_6101.jpg', 'data/sky/IMG_6061.jpg', 'data/sky/IMG_6060.jpg', 'data/sky/IMG_6100.jpg']
['data/people/IMG_6131.jpg', 'data/people/IMG_6079.jpg', 'data/people/IMG_6078.jpg', 'data/people/IMG_6130.jpg']


In [3]:
# Get train label from filepath directory
def GetTrainLabel(image_path):
    if 'sky' in image_path:
        label = 0
    elif 'earth' in image_path:
        label = 1
    elif 'people' in image_path:
        label = 2
    elif 'peace' in image_path:
        label = 3
    return label

In [4]:
# Read image as numpy array given image path
def ReadAndResizeImageAsNPArray(image_path_list, new_width=100, new_height=75):
    images_array = np.zeros(shape=(100,new_height,new_width,3))
    train_labels = np.zeros(shape=(100))
    index = 0
    for image_path in image_path_list:
        pic = Image.open(image_path)
        pic = pic.resize((new_width, new_height), Image.ANTIALIAS)
        pix = np.array(pic)
        label = GetTrainLabel(image_path)
        images_array[index] = pix
        train_labels[index] = label
        index += 1
    return images_array, train_labels
        
train_images, train_labels = ReadAndResizeImageAsNPArray(image_path_list, new_width=400, new_height=300)

In [28]:
# Testing
pic = Image.open('data/sky/IMG_6061.jpg')
pic = pic.resize((8, 6), Image.ANTIALIAS)
pix = np.array(pic)
print(pix.shape)

(6, 8, 3)


In [29]:
# Print values to check
print(train_images.shape)
print(train_labels.shape)
train_labels = train_labels.astype(int)
train_labels

(100, 300, 400, 3)
(100,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [31]:
# Randomly shuffle training data
random.seed(99999)
shuffle_index = random.sample(range(100),100)
train_images_shuffle = np.zeros(shape=(100,300,400,3))
train_labels_shuffle = np.zeros(shape=(100))
for i in range(100):
    index = shuffle_index[i]
    train_images_shuffle[i] = train_images[index]
    train_labels_shuffle[i] = train_labels[index]
print(train_images_shuffle.shape)
print(train_labels_shuffle.shape)
print(train_labels_shuffle)

(100, 300, 400, 3)
(100,)
[0. 3. 1. 2. 2. 3. 3. 1. 3. 3. 1. 2. 2. 1. 2. 0. 3. 1. 3. 1. 3. 2. 2. 0.
 0. 3. 1. 3. 3. 0. 0. 0. 2. 0. 1. 3. 2. 2. 1. 1. 0. 1. 3. 3. 2. 1. 0. 2.
 1. 0. 0. 3. 0. 2. 1. 0. 1. 3. 1. 2. 0. 0. 2. 2. 0. 2. 1. 1. 3. 1. 2. 2.
 1. 0. 3. 1. 2. 3. 0. 2. 1. 2. 3. 0. 2. 1. 3. 3. 2. 3. 0. 1. 3. 0. 3. 0.
 1. 0. 2. 0.]


In [32]:
# For plot
class_names = ['sky', 'earth', 'people', 'peace']

In [33]:
train_images = train_images / 255.0
train_images_shuffle = train_images_shuffle / 255.0

In [34]:
#create model
model = Sequential()

#add model layers
model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(300,400,3)))
model.add(Conv2D(32, kernel_size=3, activation='relu'))
model.add(Flatten())
model.add(Dense(4, activation='softmax'))

In [35]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [36]:
model.fit(train_images_shuffle, train_labels_shuffle, epochs=5)

Epoch 1/5
100/100 [==============================] - 290s 3s/step - loss: 8.8344 - acc: 0.2500
Epoch 2/5
100/100 [==============================] - 153s 2s/step - loss: 12.0886 - acc: 0.2500
Epoch 3/5
100/100 [==============================] - 133s 1s/step - loss: 12.0886 - acc: 0.2500
Epoch 4/5
100/100 [==============================] - 130s 1s/step - loss: 12.0886 - acc: 0.2500
Epoch 5/5
100/100 [==============================] - 136s 1s/step - loss: 12.0886 - acc: 0.2500
